# Preliminary items

This is based largely on [the DLC User Guide](https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/standardDeepLabCut_UserGuide.md), which provides more detail than the text offered here.

We've assumed here that DLC has been installed on your system. We'll further assume that you are acquiring 2D data from videos that include a single animal.

It is also critical that you install and an environment for DLC and that you run this code from within the environment. See the [installation instructions](https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/installation.md) for details.

As explained in the user guide, it's a good idea to trim the duration, crop the frames, compress the video a bit, and downsample the resolution of your videos to have DLC run more efficiently. See our [dlc_preprocess](dlc_preprocess.ipynb) notebook for our own command-line tricks to trim down your movies. [Mathis and Warren (2018)](https://www.biorxiv.org/content/10.1101/457242v1.abstract) details the effects of video compression and other factors on the accuracy and speed of DLC.

You'll want to adjust the parameters below for your particular computer and project.

In [ ]:
# Import essential packages
import deeplabcut as dlc
import os
from sources import dlc_help as dlch

# Customize these titles for your project 
projName            = 'geotaxis'
investigatorName    = 'mmchenry'

# Path to project directory (where dlc data will be stored)
dataPath = '/Users/mmchenry/Documents/Projects/geotaxis/dlc'

# Place all your videos for training and analysis in a directory and specify the path to that directory here
videoPath   = '/Users/mmchenry/Documents/Projects/geotaxis/Videos'

# Extension to video files
vidFileExt = 'mp4'

# Create a new project
Run the following line of code to initialize a new project. Skip this section if the project was previously created.

Note that, if you want to have all contents for the project in a single directory that may easily be copied to another computer, then set 'copy_videos=True'.

In [ ]:
# Generate list of video files in trainingVideoPath directory
pathList = dlch.list_paths(videoPath, vidFileExt)
# print(pathList) # Uncomment to display list of video files

# dlc command to create the project
dlc.create_new_project(projName, investigatorName, pathList, dataPath, copy_videos=True, multianimal=False)

# Project path
If you just created a new project, find the directories and files created in the project directory at dataPath and open config.yaml (within the dataPath directory) and copy and paste the project_path into the cell below. 

For either new or old projects, excute the cell below to define the project path and location of config.yaml for subsequent cells.

In [ ]:
project_path    = '/Users/mmchenry/Documents/Projects/geotaxis/dlc/geotaxis-mmchenry-2022-05-25' 

# Path to config file
configPath = project_path + os.path.sep + 'config.yaml'
# configPath = '/home/mmchenry/Documents/geotaxis/geotaxis-mmchenry-2022-05-25'

# Designate your list of landmarks

The config.yaml file lists key items for your project (see [this page](https://camo.githubusercontent.com/6a5086ab4a01f9eaf41e6972d86785faed0806c69a83d68a5f8b981f0cae4965/68747470733a2f2f737461746963312e73717561726573706163652e636f6d2f7374617469632f3537663664353163396637343536366635356563663237312f742f3563343066343132346437613963306232636536353163312f313534373736303731363239382f426f78312d30312e706e673f666f726d61743d3130303077) for the full glossary of parameters), including a listing of the landmarks that will be tracked (under the 'bodyparts' heading). 

Delete the entries of '- bodypart1', '- bodypart2' , etc. and key in your own landmarks (e.g., '- nose', '- eye', '- tail').

The DLC documentation suggests that it is valuable to link the landmarks with a 'skeleton'. This will also include lines that link together these landmarks in the visualization tools later. You specify a skeleton with linked points under 'skeleton:', for example a link between the nose and eye would be specified like this:

    - - nose
      - eye

There are a variety of other parameters that will come up as we proceed through the workflow, so keep track of the location of your config.yaml file.

# Extract frames
This section copies frames from your videos for you to annotate with landmark positions.

We have configured the 'extract_frames' function with the defaults. 

Here, we are using the 'kmeans' algorithm to select and downsample frames. This neglects color information, so you would want to enter algo='uniform' if you think color is helpful (though this takes more computational time). Also, you will want to configure things differently if you are doing a supplemental round of training.

It is possible to manually select frames and to exert more control through other parameters, as explained in the user guide. 

Frame extraction might take some time to complete. Copies of the frames are save to directories named after each video, in the 'labeled-data' directory. Browse to those directories to see the frames that will be used for the training.

As a general rule of thumb 100 - 200 labeled frames may be sufficient to train your network, although it never hurts to use more frames. In config.yaml, the 'numframes2paick' parameter sets the number of frames extracted from each video in your project. So, you might adjust that number based on how many videos you have in your project to aim for ~200 frames.


<!-- The choice of which training network to use is another factor. The different options are described [here](https://deeplabcut.github.io/DeepLabCut/docs/recipes/nn.html). The default (ResNet-50) is a good place to start, if you have an expensive GPU and are not sure if your videos are going to offer a big challenge. The MobileNets are worth considering if you're running DLC on a CPU. You can try different networks later. -->

<!-- There is the ability to specify a region-of-interest in the videos, but we haven't had much luck getting that to work and it's not a bad idea to crop the videos ahead of runnign DLC to reduce their file size anyway.  DLC is much more time-efficient with videos of smaller file size. -->

<!-- 'TrainingFraction' is the (number of training videos) / (total number of videos), so you might adjust this based on how many training videos you want to work through. --> 

<!-- On an initial training, shoot for 100 - 200 frames for training. If 'extract_frames' reports the number of frames being extracted outside that range, adjust the 'TrainingFraction' and 'numframes2pick' parameters in config.yaml, delete the contents of the 'labeled-data' directory, and run it again. -->

In [ ]:
# Run command to extract frames for landmark tracking
dlc.extract_frames(configPath, mode='automatic', algo='kmeans', userfeedback=False, crop=False)

# Label frames

Here, we're using the parameter defaults, which assume that we're not training for 3D experiments or tracking multiple animals.

Try launching the GUI for landmark selection by running the cell below.

In [ ]:
# %gui wx # You can try to launch the GUI with this command
dlc.label_frames(configPath)

If this command fails to launch the GUI on your system, as it did mine, then you will want to run it at the [command line](/docs/command_line.md).

Once you get the GUI launched, I recommend reading the "(D) Label Frames" section of [the DLC User Guide](https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/standardDeepLabCut_UserGuide.md) to get oriented on how the GUI works. The general idea is to pick off the coordinates of landmarks among all extracted frames from your videos. Skip on selecting coordinates for any landmarks that are not visible.

The coordinates are stored in each directory that holds the extracted frames, within 'labeled-data' in two files (csv and h5) that start with 'CollectedData'. If you load a set of images using label_frames, it is the data in the h5 file that is plotted on the frames, not the csv version. So, I suspect that the csv version is added there to offer a human-readable version of the data that is not used later in the training.

# Check annotated frames

Run this to check the frame annotation. It creates a version of each labeled frame in directories that end in '_labeled' in the 'labeled-data' directory. Those images will serve as a check on the annotation, in case the training does not go well.

In [ ]:
dlc.check_labels(configPath, visualizeindividuals=False)

# Transfer the project to a desktop machine

If you are running dlc on a laptop up to this point, it may make sense to transfer the project over to a different computer at this point. See [this doc](/docs/proj_transfer) for details.

# Create training dataset

This portion needs to be run on the machine that is going to perform the training. 
Note that trainings may be performed multiple times for the same project. 
Each 'iteration' of training has the potential to improve the neural network by building on the weightings of a prior training with corrections or additional training data. 
For each iteration, you will run the function 'create_training_dataset', which sets up a training iteration in a few steps and allows you to specify some parameters about the training, like the network type. 
For our present purposes, we'll only specify the type of augmenter because that's what they recommend in the [User Guide](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html), which means the default network type will be used (net_type= 'resnet_50').

<!-- ## Products of create_training_dataset

Within the 'training-datasets' directory, a subdirectory will be created for each iteration, which holds another subdirectory, which holds a copy of annotated coordinates for all videos in .csv and .h5 formats. 
In addition, .pickle and .mat files are stored there. The User Guide says that the .mat file  "is used by the feature detectors" and the .pickle file "contains the meta information about the training dataset." 
Not really sure what all that means, but we'll move on for now.

In the 'dlc-models' directory, a similar hierarchy of directories will be created and the directory created for each iteration will hold 'train' and 'test' directories. 
Both store parameter values within a pose_cfg.yaml file. 
In the 'train' directory, that file can be directly edited to alter the training. 
We do not need to worry about that on this first pass, but I'm merely pointing out some of the files generated for future reference. -->







In [ ]:
dlc.create_training_dataset(configPath, augmenter_type='imgaug')

# Train the network

On this first pass, we'll use all parameter defaults for running the training.



In [ ]:
dlc.train_network(configPath,maxiters=30000)